# Imports

In [1]:

from ultralytics import YOLO
from PIL import Image
import numpy as np
import pandas as pd
import os
import cv2
import json
from ultralytics.yolo.utils.plotting import Annotator

# Introduction

The goal of this project is to train a YOLO v8 model for object detection of key electrical components on GPU printed circuit boards (PCBs).

The dataset consists of 100 custom annotated images. The images are sourced from [TechPowerUp's GPU Reviews](https://www.techpowerup.com/review/?category=Graphics+Cards&manufacturer=&pp=25&order=date), with the goal to keep balance between GPU chip manufacturer (Nvidia, AMD, and Intel), and GPU board partners (Gigabyte, ASUS, MSI, Powercolor, Sapphire, etc). There are also reference design PCBs included within the images. Another balance goal was between lower end cards and higher end cards across several generations from Pascal and Polaris GPUs launched in 2016 to the latest Alchemist, Ada Lovelace and RDNA3 GPUs launched within the past 2 years. The list primarily covers gaming-focused graphics cards, although a few workstation/server models are also included such as the reference model Nvidia A2000.

The components to be detected are:
- Display output
- Power connector
- MOSFET (Also includes other drivers such as Smart Power Stages)
- Inductor
- Capacitor (Includes ceramic capacitors, tantalum capacitors and aluminum electrolytic capacitors)
- Video Memory (VRAM - GDDR and HBM)
- Core/Chip

A full breakdown of the manufacturers, models, and architecture can be found in the appendices.

# EDA

To begin, some sample images from the training set can be shown to demonstrate the annotated dataset.

A training sample (Nvidia A2000) without annotations can be shown below:

![Nvidia A2000 No Annotations](https://raw.githubusercontent.com/jl8771/msds/main/Yolo%20VRM/samples/a2000_reference.jpg)

And with annotations:

![Nvidia A2000 Annotated](https://raw.githubusercontent.com/jl8771/msds/main/Yolo%20VRM/samples/a2000_reference_annotated.jpg)

- Display outputs are shown in blue
- No power connector exists on this card
- Red shows MOSFETS, controllers and power stages
- Light blue is for inductors
- Capacitors are shown in pink
- The VRAM modules are shown in yellow
- The core is shown in the center in brown

There a wide variety of dimensions of PCBs, with some that are much denser such as in this example:

![ASUS RTX 4090 Strix](https://raw.githubusercontent.com/jl8771/msds/main/Yolo%20VRM/samples/asus_4090_strix.jpg)

In the above example, it is also possible see on both sides of the GPU core that the inductors can be rotated from the standard orientation.

Other examples are significantly noisier, including more visible components. This can be seen in the right side of the following example:

![Gigabyte RTX 2080 Gaming OC](https://raw.githubusercontent.com/jl8771/msds/main/Yolo%20VRM/samples/gigabyte_2080_gamingoc.jpg)

As seen in these examples, the component with the most instances are capacitors, followed by mosfets and inductors. These components appear the most across the 100 training images. The components with the lowest instances are core (should only appear once per image for 100 total instances), power connectors (one or two, sometimes three per image), and display connector.

| Component         | Annotations | Average Per Image |
|-------------------|-------------|-------------------|
| Capacitor         | 2418        | 24.18             |
| MOSFET            | 2208        | 22.08             |
| Inductor          | 1839        | 18.39             |
| VRAM              | 708         | 7.08              |
| Display Connector | 375         | 3.75              |
| Power Connector   | 150         | 1.5               |
| Core              | 100         | 1                 |

# Model

The model selected for this project was You Only Look Once (YOLOv8), as it one of the best models available for object detection. The first iterations of model building for this project were completed using a separate notebook loading the model in Keras for hyperparameter search and selection. Model training was completed in a separate notebook, with 16 total training runs. The model training was done on a local GPU for 100 epochs.

The notebook used to train the final version of the model used for this is available on github.

The model architecture is shown below:

In [2]:
model = YOLO('best.pt')
model.info()

YOLOv8n summary: 225 layers, 3012213 parameters, 0 gradients


(3012213, 0.0)

In [3]:
model.model

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    

Create n unique colors for visual clarity on each detected object.

In [4]:
def calc_distance(p0, p1):
    sums = []
    if len(p0) == len(p1):
        for i, (v0, v1) in enumerate(zip(p0, p1)):
            sums.append(np.power(v1 - v0,2))
    return np.sqrt(sum(sums))

def get_color_list(n):
    # Set threshold for color uniqueness, so there are not too many similar colors
    threshold = 100
    if n > 20:
        threshold = 25
    elif n > 400:
        threshold = 10
    # Default values
    rgb_dict = {0: (255, 0, 0),1: (0, 255, 0),2: (0, 0, 255),3: (255, 255, 0),4: (255, 0, 255),5: (0, 255, 255)}
    # Use default values if possible
    if n <= len(rgb_dict):
        return {k:v for (k,v) in rgb_dict.items() if k < n}
    # If not possible to only use default values, start generating new colors
    count = len(rgb_dict)
    # Fix random seed
    np.random.seed(12345)
    failed_tries = 0
    while len(rgb_dict) < n:
        if failed_tries > 25000:
            break
        rgb = tuple(np.random.randint(0, 255, 3).reshape(1, -1)[0]) #Generate random rgb value
        rgb = tuple(int(x) for x in rgb) #Convert from np.int to int
        values = list(rgb_dict.values())
        failed = False
        for v in values: #Check if color is similar to existing color, within threshold
            if calc_distance(rgb, v) < threshold:
                failed = True
        if not failed:
            rgb_dict[count] = rgb
            count += 1
        else:
            failed_tries += 1
    print(f'Generated {len(rgb_dict)} colors')
    return rgb_dict

rgb_dict = get_color_list(len(model.names))

Generated 7 colors


Make predictions for each image in validation folder, save an annotated version of the image.

In [5]:
filenames = os.listdir('./validation/')
filenames = [x for x in filenames if 'annotated' not in x]
for filename in filenames:
    results = model.predict('./validation/' + filename)

    img = cv2.imread('./validation/' + filename)

    annotator = Annotator(img)
    for i in range((results[0].boxes.data.shape[0])):
        b = results[0].boxes.xyxy[i]
        c = int(results[0].boxes.cls[i])
        d = np.round(float(results[0].boxes.conf[i]), 2)
        #annotator.box_label(b, f'{model.names[c]} {d}', rgb_dict[c])   #Annotate with name and confidence score
        #annotator.box_label(b, f'{model.names[c]}', rgb_dict[c])       #Annotate with name only
        annotator.box_label(b, f'{d}', rgb_dict[c])                     #Annotate with confidence scores only
        #annotator.box_label(b, '', rgb_dict[c])                        #Annotate with bounding box only no text

    frame = annotator.result()
    filename = filename.split('.')[0]
    cv2.imwrite('./validation/' + filename + '_annotated.jpg', frame)


image 1/1 F:\VRM\validation\680_founders.jpg: 288x640 2 display outputs, 19 mosfets, 7 inductors, 26 capacitors, 8 vrams, 1 core, 11.3ms
Speed: 2.0ms preprocess, 11.3ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 F:\VRM\validation\a2000_reference.jpg: 288x640 4 display outputs, 10 mosfets, 7 inductors, 14 capacitors, 6 vrams, 1 core, 6.5ms
Speed: 1.0ms preprocess, 6.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 F:\VRM\validation\asus_4090_strix.jpg: 416x640 4 display outputs, 1 power connector, 34 mosfets, 36 inductors, 49 capacitors, 12 vrams, 1 core, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 F:\VRM\validation\asus_6750xt_strixoc.jpg: 320x640 4 display outputs, 2 power connectors, 28 mosfets, 19 inductors, 15 capacitors, 6 vrams, 1 core, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 

# Results

The results per epoch can be seen in these plots below:

![Results](https://raw.githubusercontent.com/jl8771/msds/main/Yolo%20VRM/results/results.png)

| Class           | Images | Instances | Box(P | R     | mAP50 | mAP50-95) |
|-----------------|--------|-----------|-------|-------|-------|-----------|
| all             | 100    | 12352     | 0.962 | 0.608 | 0.706 | 0.566     |
| display output  | 100    | 595       | 0.966 | 0.632 | 0.745 | 0.566     |
| power connector | 100    | 230       | 0.952 | 0.635 | 0.739 | 0.631     |
| mosfet          | 100    | 3460      | 0.926 | 0.53  | 0.634 | 0.413     |
| inductor        | 100    | 2977      | 0.97  | 0.597 | 0.672 | 0.522     |
| capacitor       | 100    | 3811      | 0.94  | 0.592 | 0.67  | 0.486     |
| vram            | 100    | 1122      | 0.993 | 0.631 | 0.709 | 0.639     |
| core            | 100    | 157       | 0.99  | 0.64  | 0.773 | 0.704     |

Speed: 0.8ms preprocess, 1.4ms inference, 0.0ms loss, 17.2ms postprocess per image

The confusion matrix can be seen below:

![Confusion Matrix](https://raw.githubusercontent.com/jl8771/msds/main/Yolo%20VRM/results/confusion_matrix.png)

From the confusion matrix, there are a significant number of mosfets, capacitors and inductors labeled as background by the model.

The F1-confidence curve can be seen below:

![F1](https://raw.githubusercontent.com/jl8771/msds/main/Yolo%20VRM/results/F1_curve.png)

MOSFET, Capacitor and inductor have the poorest F1 score performance.

Core has consistent F1 score across all confidence levels.

# Discussion

The mAP50 could be improved, especially for the mosfet and capacitor classes. This is likely because all three of these classes contain subcategories.

Capacitors include different types of capacitors, including ceramic capacitors, tantalum capacitors and aluminum electrolytic capacitors which have different appearances and packaging. They also only contain large capacitors, smaller capacitors are not included. MOSFETs actually contain MOSFETs, smart power stages, controllers and other similarly-packaged components. By including these subcategories as specifically marked subcategories in their own distinct class, performance for these classes should be able to be improved.

Display output, power connector could also be subcategorized for the specific type or wattage, respectively.

# Appendix

Breakdown of distribution of card manufacturers, architectures, etc


In [10]:
df = pd.read_csv('VRMs.csv')
df.groupby(['Card_Manufacturer', 'Silicon_Manufacturer'])['Model'].count()

Card_Manufacturer  Silicon_Manufacturer
AMD                AMD                     11
ASUS               AMD                      7
                   Nvidia                   7
AsRock             AMD                      8
Colorful           Nvidia                   1
EVGA               Nvidia                   6
Gainward           Nvidia                   2
Galax              Nvidia                   3
Gigabyte           AMD                      4
                   Nvidia                   9
Intel              Intel                    3
MSI                AMD                      5
                   Nvidia                   7
Nvidia             Nvidia                   9
PNY                Nvidia                   2
Palit              Nvidia                   3
Powercolor         AMD                      5
Sapphire           AMD                      7
XFX                AMD                      5
Zotac              Nvidia                   6
Name: Model, dtype: int64

In [11]:
df.groupby(['Series'])['Model'].count()

Series
Ada Lovelace        24
Alchemist            3
Ampere              15
Arctic Islands       2
Kepler               1
Maxwell              1
Pascal               6
Polaris              4
RDNA1                8
RDNA2               22
RDNA3               12
Southern Islands     1
Turing               8
Vega                 2
Vega II              1
Name: Model, dtype: int64

In [12]:
df.groupby(['Model'])['Model'].count()

Model
1050Ti        1
1070          2
1080          2
1630          1
1650          2
2060          1
2070Super     3
2080          1
3050          4
3060Ti        2
3070Ti        1
3080          2
3080Ti        2
3090          1
3090Ti        2
4060          1
4060Ti        3
4070          4
4070Ti        3
4080          6
4090          7
470           1
480           1
5500          1
5500XT        1
5600XT        4
5700XT        2
580           1
590           3
6400          1
6500XT        1
6600          2
6600XT        4
6700XT        3
6750XT        2
680           1
6800XT        3
6900XT        2
6950XT        4
7600          5
7900XT        2
7900XTX       5
980Ti         1
A2000         1
A380          1
A750          1
A770          1
HD7850        1
Radeon VII    1
Titan         1
Vega56        1
Vega64        1
Name: Model, dtype: int64